<a href="https://colab.research.google.com/github/guyfloki/ai-image-detector/blob/main/ai_image_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DOWNLOAD** **ZIP** **AND** **CSV**

In [1]:
#If it does not downloading due to high traffic on the file, please use directly link.
#Train.zip
!gdown --id 1-1ddgedsRSvJm3ERQwJPy4tq4cB0uWe9

^C


C:\Users\stone\.conda\envs\image-detector\lib\site-packages\gdown\__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Error:

	HTTPSConnectionPool(host='drive.google.com', port=443): Max retries
	exceeded with url: /uc?id=1-1ddgedsRSvJm3ERQwJPy4tq4cB0uWe9 (Caused by
	ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at
	0x00000235E1FAFC40>, 'Connection to drive.google.com timed out.
	(connect timeout=None)'))

To report issues, please visit https://github.com/wkentaro/gdown/issues.


In [ ]:
#Test.zip
!gdown --id 1-1xneYPH9fgSPCVnlZrhCM6c0FpFEl6B

In [ ]:
#Train.csv
!gdown --id 1rM2r7cxve7ApXCHTlBnMyD50n5hfnoAX

In [ ]:
#Test.csv
!gdown --id 1-GzzsszBlrmUHaDvoqVJgFfLvzRQDaBV

**UNZIP** **FILES**

In [ ]:
!unzip /content/Train.zip -d /content/

In [ ]:
!unzip /content/Test.zip -d /content/Test

**CHECK** **SYSTEM** **DETAILS**

In [ ]:
!cat /proc/cpuinfo

In [2]:
!nvidia-smi

Sat Mar 23 19:43:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.41                 Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti    WDDM | 00000000:01:00.0  On |                  N/A |
|  0%   33C    P8               23W / 250W|   1553MiB / 11264MiB |     11%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**MAIN** **PROCESS**

In [ ]:
!pip install --upgrade transformers

In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision import transforms
from tqdm import tqdm
from transformers import get_linear_schedule_with_warmup
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer, AutoImageProcessor, EarlyStoppingCallback,AutoFeatureExtractor
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, cohen_kappa_score, log_loss
from torch.cuda.amp import autocast, GradScaler
import glob
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoFeatureExtractor, CvtForImageClassification
from safetensors.torch import load_file

**PREPARE** **DATASET**

In [ ]:
# Data preparation
class CustomDataset(Dataset):
    def __init__(self, csv_path, transform=None):
        self.data = pd.read_csv(csv_path)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.loc[idx, 'image_path']
        image = Image.open(img_path).convert("RGB")
        label = int(self.data.loc[idx, 'target'])

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
def compute_class_weights(n_samples_class0, n_samples_class1):
    total = n_samples_class0 + n_samples_class1
    weight_class0 = total / (2 * n_samples_class0)
    weight_class1 = total / (2 * n_samples_class1)
    return weight_class0, weight_class1

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
train_data = CustomDataset("/content/train.csv", transform=transform)
test_data = CustomDataset("/content/test.csv", transform=transform)

In [ ]:
n_samples_class0 = sum(train_data.data["target"] == 0.0)
n_samples_class1 = sum(train_data.data["target"] == 1.0)
weight_class0, weight_class1 = compute_class_weights(n_samples_class0, n_samples_class1)

In [ ]:
samples_weights = [weight_class0 if label == 0.0 else weight_class1 for label in train_data.data["target"]]
weighted_sampler = WeightedRandomSampler(samples_weights, len(samples_weights), replacement=True)

In [ ]:
train_loader = DataLoader(train_data, batch_size=128, sampler=weighted_sampler, num_workers=6, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False, num_workers=6, pin_memory=True)

**MODEL** **DEFINITION**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor([weight_class0, weight_class1]).to(device))

In [ ]:
model = CvtForImageClassification.from_pretrained('microsoft/cvt-13')

In [ ]:
class CustomClassifier(nn.Module):
    def __init__(self):
        super(CustomClassifier, self).__init__()
        # First Hidden Layer
        self.fc1 = nn.Linear(384, 256)
        self.mish1 = nn.Mish(inplace=False)
        self.norm1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(p=0.5)

        # Second Hidden Layer
        self.fc2 = nn.Linear(256, 128)
        self.mish2 = nn.Mish(inplace=False)
        self.norm2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(p=0.3)

        # Output Layer
        self.fc_out = nn.Linear(128, 2)

    def forward(self, x):
        x = self.dropout1(self.norm1(self.mish1(self.fc1(x))))
        x = self.dropout2(self.norm2(self.mish2(self.fc2(x))))
        x = self.fc_out(x)
        return x


model = model.to(device)
model.classifier = CustomClassifier().to(device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999), weight_decay=1e-5, eps=1e-8)

In [ ]:
scaler = GradScaler()

**MAIN** **TRAINING**

In [ ]:
#change path for your own
list_of_files = glob.glob('/content/drive/MyDrive/CVT-13_2/model_epoch_*.pth')
if list_of_files:  # Check if the list is not empty
    # Identify the latest model
    latest_file = max(list_of_files, key=os.path.getctime)

    # Load the latest model
    checkpoint = torch.load(latest_file)
    train_losses = checkpoint['train_losses']
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    starting_epoch = checkpoint['epoch'] + 1
    scaler.load_state_dict(checkpoint['scaler_state_dict'])
    for g in optimizer.param_groups:
        g['lr'] = 1e-5
    # Load the average loss
    avg_loss_loaded = checkpoint.get('avg_loss', None)  # Use None if avg_loss is not found

    model.train()  # or model.eval() if you are doing evaluation instead of training
    total_epochs = 50
else:
    # If no saved model, start from epoch 1
    total_epochs = 50
    starting_epoch = 0
    avg_loss_loaded = None
    train_losses = []

In [ ]:
model = model.to(device)

In [ ]:
for g in optimizer.param_groups:
  print(g['lr'])

1e-05


In [ ]:
for epoch in range(starting_epoch, total_epochs):
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch+1}, Learning Rate: {current_lr:.6f}")

    model.train()
    total_loss = 0.0

    with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}", unit="batch") as progress_bar:
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs.logits, labels)

            # Gradient clipping
            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            progress_bar.set_postfix({"Loss": loss.item()})
            progress_bar.update()

            # Free up memory
            del inputs, labels, outputs

    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    print(f"Epoch {epoch+1} - Average Training Loss: {avg_train_loss:.4f}")

    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'avg_loss': avg_train_loss,
        'scaler_state_dict': scaler.state_dict(),
        'train_losses': train_losses,
        #change path for your own
    }, f"/content/drive/MyDrive/CVT-13_2/model_epoch_{epoch}.pth")

    torch.cuda.empty_cache()

**TEST**

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
# Assuming your training loop code ends before this
model.eval()
test_loss = 0
all_predicted = []
all_labels = []

with torch.no_grad():  # Disables gradient calculation for evaluation, which reduces memory usage
    with tqdm(total=len(test_loader), desc="Evaluation", unit="batch") as progress_bar:
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            with autocast():  # Use autocast if you're evaluating with mixed precision
                outputs = model(inputs)
                loss = criterion(outputs.logits, labels)  # Assuming your model outputs logits

            test_loss += loss.item()
            _, predicted = outputs.logits.max(1)  # Get the index of the max log-probability
            all_predicted.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            progress_bar.set_postfix({"Test Loss": loss.item()})
            progress_bar.update()

# Convert all_labels and all_predicted to numpy arrays if they are not already
all_labels = np.array(all_labels)
all_predicted = np.array(all_predicted)

avg_test_loss = test_loss / len(test_loader)
accuracy = accuracy_score(all_labels, all_predicted)
precision = precision_score(all_labels, all_predicted, average='macro')
recall = recall_score(all_labels, all_predicted, average='macro')
f1 = f1_score(all_labels, all_predicted, average='macro')

print(f'Average Test Loss: {avg_test_loss:.4f}')
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')


Evaluation: 100%|██████████| 1978/1978 [07:54<00:00,  4.17batch/s, Test Loss=1.33e-5]


Average Test Loss: 0.1275
Accuracy: 98.54%
Precision: 0.99
Recall: 0.98
F1 Score: 0.98


In [ ]:
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt

# Calculate the confusion matrix
cm = confusion_matrix(all_labels, all_predicted)

# Plot the confusion matrix
fig, ax = plot_confusion_matrix(conf_mat=cm,
                                show_absolute=True,
                                show_normed=False)  # You can set show_normed to True to show percentages
plt.title('Confusion Matrix')
plt.show()
plt.savefig("confusion_matrix.png")

In [ ]:
import matplotlib.pyplot as plt

# Assuming train_losses is a list that contains the average loss of each epoch
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss Over Time')
plt.legend()
plt.show()
plt.savefig("loss.png")

**MAKE** **PREDICTION**

In [ ]:
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import requests
from io import BytesIO

# Function to load an image from a URL
def load_image_from_url(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    return img

# URL of your custom image
image_url = ""

# Load and transform your custom image
custom_image = load_image_from_url(image_url)
transformed_image = transform(custom_image).unsqueeze(0)  # Add batch dimension
transformed_image = transformed_image.to(device)

# Evaluate the custom image using the model
model.eval()
with torch.no_grad():  # Disables gradient calculation for evaluation
    # If you're using autocast for mixed precision
    with torch.cuda.amp.autocast():
        outputs = model(transformed_image)
        # Use the logits attribute to get the prediction scores
        logits = outputs.logits
        _, predicted = logits.max(1)  # Get the index of the max log-probability

# Print the prediction
print(f'Predicted class: {predicted.item()}')

# If you want to get the probabilities
probabilities = torch.nn.functional.softmax(logits, dim=1)
print(f'Class probabilities: {probabilities}')
